In [ ]:
# ====================================================================
# SEL 1: Import Library dan Inisialisasi Parameter
# ====================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from math import factorial # Diperlukan untuk rumus teoritis M/M/c

warnings.filterwarnings('ignore')
np.random.seed(42) 

# --- PARAMETER KASUS SUPERMARKET M/M/c ---
l = 4.0  # (Lambda): Laju kedatangan (Rata-rata 4.0 pelanggan/menit)
µ = 1.5  # (Mu): Laju pelayanan per server (Rata-rata 1.5 pelanggan/menit)
c = 5    # Jumlah Server (Kasir)
N_CUSTOMERS = 3000 # Jumlah pelanggan yang disimulasikan

# Cek Stabilitas
rho_sistem = l / (c * µ)
print(f"--- SISTEM ANTRIAN KASIR SUPERMARKET M/M/{c} ---")
print(f"Laju Kedatangan (λ): {l} pelanggan/menit")
print(f"Laju Pelayanan (µ) per server: {µ} pelanggan/menit")
print(f"Jumlah Server (c): {c}")
print(f"Rasio Trafik Sistem (ρ): {rho_sistem:.4f} (Utilitas sekitar 53.33%)")

if rho_sistem >= 1:
    print("Sistem tidak stabil (λ >= cµ). Ubah parameter!")

In [ ]:
# ====================================================================
# SEL 2: Perhitungan Teoritis M/M/c
# ====================================================================

# Perhitungan Teoritis Berdasarkan Rumus M/M/c
rho = rho_sistem 

# 1. Hitung Probabilitas Sistem Kosong (P0)
sum_term = 0
for n in range(c):
    sum_term += (l/µ)**n / factorial(n)

P0 = 1 / (sum_term + (1/factorial(c)) * (l/µ)**c * (1/(1-rho)))

# 2. Hitung Rata-rata Panjang Antrian (Lq)
Lq_teori = (P0 * (l/µ)**c * rho) / (factorial(c) * (1-rho)**2)

# 3. Hitung Rata-rata Waktu Tunggu (Wq)
Wq_teori = Lq_teori / l

# 4. Hitung Rata-rata Waktu di Sistem (Ws)
Ws_teori = Wq_teori + (1/µ)

# 5. Hitung Rata-rata Panjang Sistem (Ls)
Ls_teori = Lq_teori + (l/µ)

print("\n--- Hasil Teoritis M/M/c Supermarket ---")
print(f"P0 (Prob. Sistem Kosong): {P0:.4f}")
print(f"Ls (Panjang Sistem): {Ls_teori:.4f} pelanggan")
print(f"Lq (Panjang Antrian): {Lq_teori:.4f} pelanggan")
print(f"Ws (Waktu Sistem): {Ws_teori:.4f} menit")
print(f"Wq (Waktu Tunggu): {Wq_teori:.4f} menit")

In [ ]:
# ====================================================================
# SEL 3: Inti Simulasi M/M/c (Perhitungan Waktu)
# ====================================================================

# 1. Membangkitkan Bilangan Acak
inter_arrival_times = np.random.exponential(1/l, size=N_CUSTOMERS)
service_times = np.random.exponential(1/µ, size=N_CUSTOMERS)
arrival_times = np.cumsum(inter_arrival_times)

# 2. Inisialisasi Variabel M/M/c
# server_finish_times: Waktu kapan setiap server akan menjadi bebas
server_finish_times = np.zeros(c) 
finish_times = np.zeros(N_CUSTOMERS)
start_times = np.zeros(N_CUSTOMERS)

for i in range(N_CUSTOMERS):
    
    current_arrival = arrival_times[i]
    
    # Cari server yang paling cepat bebas
    server_idx = np.argmin(server_finish_times) 
    server_available_time = server_finish_times[server_idx]
    
    # Start Time = MAX(Waktu Pelanggan Datang, Waktu Server Paling Cepat Bebas)
    start_times[i] = max(current_arrival, server_available_time)
    
    # Waktu Selesai = Waktu Mulai + Waktu Pelayanan
    current_service = service_times[i]
    finish_times[i] = start_times[i] + current_service
    
    # Update waktu server yang digunakan (server itu baru bebas pada finish_times[i])
    server_finish_times[server_idx] = finish_times[i]

# 3. Menghitung Metrik Kinerja Pelanggan Individual
data = {
    'arrival_times': arrival_times,
    'service_times': service_times,
    'start_times': start_times,
    'finish_times': finish_times,
}
df = pd.DataFrame(data)

df['total_times'] = df['finish_times'] - df['arrival_times'] # Ws
df['wait_times'] = df['start_times'] - df['arrival_times']   # Wq

In [ ]:
# ====================================================================
# SEL 4: Analisis dan Perbandingan Hasil
# ====================================================================

# 1. Menghitung Waktu Rata-rata dari Simulasi
Ws_simulasi = df['total_times'].mean()
Wq_simulasi = df['wait_times'].mean()

# 2. Menghitung Panjang Rata-rata dari Simulasi (Hukum Little: L = λW)
Ls_simulasi = l * Ws_simulasi
Lq_simulasi = l * Wq_simulasi

print("\n--- Hasil Analisis Simulasi Kasir Supermarket ---")
print(f"Rata-rata Waktu di Sistem (Ws): {Ws_simulasi:.4f} menit")
print(f"Rata-rata Waktu Tunggu (Wq): {Wq_simulasi:.4f} menit")
print(f"Rata-rata Panjang Sistem (Ls): {Ls_simulasi:.4f} pelanggan")
print(f"Rata-rata Panjang Antrian (Lq): {Lq_simulasi:.4f} pelanggan")

print("\n--- Perbandingan Teoritis vs Simulasi ---")
comparison = pd.DataFrame({
    'Metrik': ['Ls', 'Lq', 'Ws', 'Wq'],
    'Teoritis': [Ls_teori, Lq_teori, Ws_teori, Wq_teori],
    'Simulasi': [Ls_simulasi, Lq_simulasi, Ws_simulasi, Wq_simulasi]
})
comparison['Error (%)'] = (abs(comparison['Simulasi'] - comparison['Teoritis']) / comparison['Teoritis']) * 100
print(comparison.to_markdown(index=False, floatfmt=".4f"))

In [ ]:
# ====================================================================
# SEL 5: Visualisasi dan Menyimpan File PNG (Output Wajib 2)
# ====================================================================

# 1. Visualisasi Distribusi Waktu Tunggu (Wq)
plt.figure(figsize=(10, 5))
sns.histplot(df['wait_times'], bins=30, kde=True, color='purple')
plt.axvline(Wq_simulasi, color='red', linestyle='--', label=f'Wq Simulasi: {Wq_simulasi:.4f} mnt')
plt.axvline(Wq_teori, color='black', linestyle=':', label=f'Wq Teoritis: {Wq_teori:.4f} mnt')
plt.title(f'Distribusi Waktu Tunggu Pelanggan di Kasir Supermarket M/M/{c}')
plt.xlabel('Waktu Tunggu di Antrian (menit)')
plt.ylabel('Frekuensi')
plt.legend()
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
# Menyimpan visualisasi pertama sebagai file PNG
plt.savefig(f'visualisasi_distribusi_wq_supermarket_mmc{c}.png') 
plt.show()

# 2. Visualisasi Waktu di Sistem (Ws) per Pelanggan (untuk melihat tren)
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['total_times'], alpha=0.6, label='Ws Individual')
plt.axhline(Ws_simulasi, color='red', linestyle='--', label=f'Ws Simulasi Rata-rata: {Ws_simulasi:.4f} mnt')
plt.axhline(Ws_teori, color='green', linestyle=':', label=f'Ws Teoritis Rata-rata: {Ws_teori:.4f} mnt')
plt.title(f'Waktu Pelanggan di Sistem (Ws) Supermarket M/M/{c} Berdasarkan Urutan Kedatangan')
plt.xlabel('Urutan Pelanggan (ke-N)')
plt.ylabel('Waktu di Sistem (menit)')
plt.legend()
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
# Menyimpan visualisasi kedua sebagai file PNG
plt.savefig(f'visualisasi_tren_ws_supermarket_mmc{c}.png')
plt.show()

print("\nVisualisasi telah selesai dan file PNG telah disimpan.")